# VM translator based on elements of computing systems

In [20]:
import sys, os
from io import StringIO
from typing import List, Dict, Tuple
from enum import Enum

# sys.path.append(os.path.join(os.path.dirname(__file__), "..", "lib"))
from lib.utils import pretty_format_dict

CommandType = Enum(
    'CommandType', 
    ['C_ARITHMETIC', # page 130, fig 7.5: e.g., add, sub, neg ...
     'C_PUSH', # page 131: push <segment> index, e.g., push argument 0 // stack.push(argument[0])
     'C_POP', # page 131: pop <segment> index, e.g., pop argment 0 // argment[0] = stack.pop()
     'C_LABEL', # page 159: label symbol, marks location in code, scope is within the function
     'C_GOTO',  # page 159: goto label, unconditional jump
     'C_IF', # page 159: if-goto label, pc = label if stack.pop() == 0 else pc + 1, label must be within the same function
     'C_FUNCTION', # page 163: function f k, where k is num local variables
     'C_RETURN', # page 163: return, return control to the caller
     'C_CALL', # page 163: call f n, where f is a function and n is number of arguments
    ]
)

ARITHMETIC_COMMANDS = ['add', 'sub', 'neg', 'eq', 'gt', 'lt', 'and', 'or', 'not']

def getCommandType(command: str)->CommandType:
    if command in ARITHMETIC_COMMANDS:
        return CommandType.C_ARITHMETIC
    elif command.startswith('push'):
        return CommandType.C_PUSH
    elif command.startswith('pop'):
        return CommandType.C_POP
    elif command.startswith('label'):
        return CommandType.C_LABEL
    elif command.startswith('goto'):
        return CommandType.C_GOTO
    elif command.startswith('if-goto'):
        return CommandType.C_IF
    elif command.startswith('function'):
        return CommandType.C_FUNCTION
    elif command.startswith('return'):
        return CommandType.C_RETURN
    elif command.startswith('call'):
        return CommandType.C_CALL
    
class Interpretor:
    '''VM interpretor, essentially a simulator in python
    instead of compiling the code, just run on the fly instead
    this is like the class Machine in assembler

    Here we assume VM only contain functions
    '''
    def __init__(self, max_steps:int=100, verbose:bool=True):
        self.max_steps = max_steps
        self.verbose = verbose
        
    def load(self, vm_fnames: List[str]):
        self.machine = {
            'stack': [],
            'heap': [],
            'functions': {},
            'current_function': "Sys.init", # always start from Sys.init
            'pc': 0 # line number within current_function
        }
        for fname in vm_fnames:
            with open(fname) as f:
                self._parse_functions(f.readlines())

    def __repr__(self):
        return f'VM interpretor(\n{pretty_format_dict(self.machine)}\n)'

    def _parse_functions(self, codes: List[str]):
        # parse out all the functions into self.machine['functions']
        func_name, func_codes = None, []

        for i, l in enumerate(codes):

            # sanitize line
            l = l.strip()
            if '//' in l: # comment
                l = l[:l.index('//')].strip()
    
            # scrape function
            if getCommandType(l) is CommandType.C_FUNCTION:
                if func_name is not None:
                    self.machine['functions'][func_name] = func_codes
                    self._parse_functions(codes[i:])
                    return
                else:
                    func_name = l.split()[1]
            else:
                if l != "": func_codes.append(l)
                
        if func_name is not None:
            self.machine['functions'][func_name] = func_codes


    def advance(self)->bool:
        func_name, pc = self.machine['current_function'], self.machine['pc']
        codes = self.machine['functions'][func_name]
        if pc >= len(codes):
            # finished execution
            return False

        code = codes[pc]
        if self.verbose:
            print(f'instruction {func_name}[pc]: {code}')
            print(self)
        
        self.machine['pc'] += 1
        # instruction decoding, TODO: patch on command type
        
        # instruction execution
        
        return True

    
    def __call__(self, vm_fnames: List[str]):
        self.load(vm_fnames)
        assert self.machine['current_function'] in self.machine['functions'], f'{self.machine["current_function"]} need to be in vm code'
        steps = 0
        while steps <= self.max_steps:
            if self.verbose:
                print('Machine step', steps)
            steps += 1
            ok = self.advance()
            if not ok:
                print('Finished execution')
                return
        print(f'Program terminated b/c exceeding max step of {self.max_steps}')        

vm_interpretor = Interpretor()
vm_interpretor(['vm_codes/test.vm'])

Machine step 0
instruction Sys.init[pc]: push constant 3
VM interpretor(
{
    "stack": [],
    "heap": [],
    "functions": {
        "double": [
            "push argument 0",
            "push argument 0",
            "add",
            "return"
        ],
        "Sys.init": [
            "push constant 3",
            "pop argument 0",
            "call double 1",
            "return"
        ]
    },
    "current_function": "Sys.init",
    "pc": 0
}
)
Machine step 1
instruction Sys.init[pc]: pop argument 0
VM interpretor(
{
    "stack": [],
    "heap": [],
    "functions": {
        "double": [
            "push argument 0",
            "push argument 0",
            "add",
            "return"
        ],
        "Sys.init": [
            "push constant 3",
            "pop argument 0",
            "call double 1",
            "return"
        ]
    },
    "current_function": "Sys.init",
    "pc": 1
}
)
Machine step 2
instruction Sys.init[pc]: call double 1
VM interpretor(
{
    

In [ ]:
class Translator:
    '''VM code -> Assembly code'''
    def __init__(self):
        pass

    def __call__(self, vm_fnames: List[str])->str:
        pass

class Parser:

    def __init__(self, fname:str):
        self.fstream = open(fname, 'r')
        self.fname = fname
        self.lineno = -1 # vm line number

    def advance(self)->Tuple[bool, str, int]:
        '''
        read the next command
        return (ok, next command, reference_line_in_orig_file)
        '''
        l = self.filestream.readline()
        if l == '':
            # EOF error
            return False, '', self.lineno

        self.lineno += 1
        l = l.strip()
        # todo: maybe handle multi line comment like /* */
        if '//' in l: # comment
            l = l[:l.index('//')].strip()
            
        if l == '':    
            return self.advance()
        else:
            return True, l, self.lineno

    def commandType(self, command):
        return getCommandType(command)
        
        